[MIT License]

PyAudio is distributed under the MIT License:
Copyright (c) 2006 Hubert Pham
Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

[PIL Software License]

The Python Imaging Library (PIL) is
Copyright © 1997-2011 by Secret Labs AB
Copyright © 1995-2011 by Fredrik Lundh
By obtaining, using, and/or copying this software and/or its associated documentation, you agree that you have read, understood, and will comply with the following terms and conditions:
Permission to use, copy, modify, and distribute this software and its associated documentation for any purpose and without fee is hereby granted, provided that the above copyright notice appears in all copies, and that both that copyright notice and this permission notice appear in supporting documentation, and that the name of Secret Labs AB or the author not be used in advertising or publicity pertaining to distribution of the software without specific, written prior permission.
SECRET LABS AB AND THE AUTHOR DISCLAIMS ALL WARRANTIES WITH REGARD TO THIS SOFTWARE, INCLUDING ALL IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS. IN NO EVENT SHALL SECRET LABS AB OR THE AUTHOR BE LIABLE FOR ANY SPECIAL, INDIRECT OR CONSEQUENTIAL DAMAGES OR ANY DAMAGES WHATSOEVER RESULTING FROM LOSS OF USE, DATA OR PROFITS, WHETHER IN AN ACTION OF CONTRACT, NEGLIGENCE OR OTHER TORTIOUS ACTION, ARISING OUT OF OR IN CONNECTION WITH THE USE OR PERFORMANCE OF THIS SOFTWARE.

[3-clause BSD License]

Copyright (c) 2014-2017, Anthony Zhang <azhang9@gmail.com>
All rights reserved.
Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:
1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
3. Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

In [1]:
#  #VOICEVOXを忘れずに起動する事！
# # 各種ライブラリインストール
    
# !pip install --upgrade pip
# !pip install pyaudio    
# !pip install openai
# !pip install --upgrade openai 
# !pip show openai

# !pip install SpeechRecognition
# !pip install Pillow

# !pip install pyinstaller#実行ファイル作成用

In [2]:
#chatGPT
import openai
openai.api_key = ""

In [3]:
# #合成した音声を出力するための関数を定義（after関数）
# import pyaudio
# import wave
# import time

# def update_image(nurse_now):
#     if nurse_now == "o":canvas.create_image(-scrw/20, 0, anchor=tk.NW, image=nurse_o)
#     if nurse_now == "n":canvas.create_image(-scrw/20, 0, anchor=tk.NW, image=nurse_n)
#     canvas.update_idletasks()
#     print(nurse_now)
#     print("OK")
        
# def play_wav():
#     canvas.create_image(-scrw/20, 0, anchor=tk.NW, image=nurse_o)#一口目
#     canvas.update()
#     CHUNK = 1024

#     wf = wave.open("audio.wav", 'rb')
#     p = pyaudio.PyAudio()

#     stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
#                     channels=wf.getnchannels(),
#                     rate=wf.getframerate(),
#                     output=True)
#     data = wf.readframes(CHUNK)
#     c=0
#     nurse_prev = ""
#     while data:
#         rms = 0.0  # 瞬時の音量を計算するための変数
#         # データがバイト列として与えられるため、16ビットの整数に変換してから処理する
#         for i in range(0, len(data), 2):
#             sample = int.from_bytes(data[i:i+2], byteorder="little", signed=True)
#             rms += sample ** 2
#         rms /= CHUNK
#         rms = rms ** 0.5  # 平均平方根 (RMS)
#         if c%8 == 0:
#             print(rms)

#             if rms > 500:  # 音量が大きい場合は nurse_o を表示
#                 nurse_now = "o"
#                 #print(nurse_now)
#                 if nurse_prev != nurse_now:canvas.after(8, update_image,nurse_now)                
#             else:  # 音量が小さい場合は nurse_n を表示
#                 nurse_now = "n"
#                 #print(nurse_now)
#                 if nurse_prev != nurse_now:canvas.after(8, update_image,nurse_now)
#             #if nurse_prev != nurse_now:canvas.update()
#             nurse_prev = nurse_now
#         stream.write(data)
#         data = wf.readframes(CHUNK)
#         c+=1
#     canvas.create_image(-scrw/20, 0, anchor=tk.NW, image=nurse_n)#口閉じる
#     canvas.update()
#     stream.stop_stream()
#     stream.close()
#     p.terminate()

In [4]:
#合成した音声を出力するための関数を定義（逆転マルチスレッド）
import pyaudio
import wave
import time
import threading
import queue

# Queueオブジェクトの生成
q = queue.Queue()
q.put("n")            
def play_stream():
    CHUNK = 1024

    wf = wave.open("audio.wav", 'rb')
    p = pyaudio.PyAudio()

    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=wf.getnchannels(),
                    rate=wf.getframerate(),
                    output=True)
    data = wf.readframes(CHUNK)#１音目を発声
    c = 0
    nurse_now = "n"
    while data:
        global rms
        rms= 0.0
        for i in range(0, len(data), 2):
            sample = int.from_bytes(data[i:i+2], byteorder="little", signed=True)
            rms += sample ** 2
        rms /= CHUNK
        rms = rms ** 0.5
        if c % 4 == 0:
                #print(rms)
            if rms > 500 and nurse_now == "n":
                nurse_now = "o"
                q.put("o")
            elif rms < 1000 and nurse_now == "o":
                nurse_now = "n"
                q.put("n")
                #print(nurse_now)

        stream.write(data)
        data = wf.readframes(CHUNK)
        c += 1
    stream.stop_stream()
    stream.close()
    p.terminate()
    q.put("end")
        
def play_wav():
    nurse = canvas.create_image(-scrw/20, 0, anchor=tk.NW, image=nurse_n)
    canvas.update()
    thread = threading.Thread(target=play_stream)
    thread.start()
    while True:
        #time.sleep(0.016)
        if not q.empty():
            nurse_now = q.get()
            #if nurse_prev != nurse_now :
            if nurse_now == "n":
                canvas.create_image(-scrw/20, 0, anchor=tk.NW, image=nurse_n)
                #canvas.itemconfigure(nurse, image=nurse_n)
                #print("n")
            if nurse_now == "o":
                canvas.create_image(-scrw/20, 0, anchor=tk.NW, image=nurse_o)
                #canvas.itemconfigure(nurse, image=nurse_O)
                #nurse_now:canvas.after(8, update_image,nurse_now)  #afteerはこのコードを実行してから8ms後にuipdateが実行される
                #print("o")
            canvas.update_idletasks()
            #canvas.update()
            if nurse_now == "end":break
    canvas.create_image(-scrw/20, 0, anchor=tk.NW, image=nurse_n) 
    canvas.update_idletasks()
    thread.join()

In [5]:
#VOICEVOXのアプリにクエリを飛ばして合成してもらう関数を定義
import json
import requests
def generate_wav(text, speaker = 1, filepath='./audio.wav'):
    host = 'localhost'
    port = 50021
    params = (
        ('text', text),
        ('speaker', speaker),
    )
    response1 = requests.post(
        f'http://{host}:{port}/audio_query',
        params=params
    )
    headers = {'Content-Type': 'application/json',}
    response2 = requests.post(
        f'http://{host}:{port}/synthesis',
        headers=headers,
        params=params,
        data=json.dumps(response1.json())
    )

    wf = wave.open(filepath, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(2)
    wf.setframerate(24000)
    wf.writeframes(response2.content)
    wf.close()

In [6]:
import speech_recognition as sr

def record_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        try:
            print("話してください...")
            r.adjust_for_ambient_noise(source)  # ノイズの調整
            audio = r.listen(source, timeout=1) # 何秒の沈黙を許すか    
            print("音声認識結果: " + r.recognize_google(audio, language='ja-JP'))
            return r.recognize_google(audio, language='ja-JP')
        except sr.WaitTimeoutError:
            #print("タイムアウトしました。")
            return "タイムアウトしました。"
        except sr.UnknownValueError:
            print("音声を理解できませんでした。")
            return "音声を理解できませんでした。" 
        except sr.RequestError:
            print("音声認識サービスでエラーが発生しました。")
            return "音声認識サービスでエラーが発生しました。" 

In [7]:
messageCount=0
def UpdateMessageLog (text, tag):
    global messageCount
    messageCount+=1
    if messageCount>2:
        canvas.move("input", 0, -scrh/4)#会話履歴を上にずらす
        canvas.move("res", 0, -scrh/4)
#    if len(text) > 39 : text = text[:37] + "..."#文字を切るLinux用
    if len(text) > 60 : text = text[:58] + "..."#文字を切るWindows用
    if tag == "input":
        if messageCount==1:
            imagey=scrh*1/4
            #print("OK")
        else:
            imagey=scrh*2/4
        canvas.create_image(scrw*77/100, imagey, anchor=tk.N, image=black_box,  tag=tag)
        canvas.create_text(
            scrw*77/100,
            imagey+scrh*3/100,
            text=text[:15] + "\n" + text[15:30] + "\n" + text[30:45] + "\n" + text[45:60],
            fill="white",
            font=("HG丸ｺﾞｼｯｸM-PRO", canvas.winfo_height()//40, "bold"),
            anchor="n",
            tag=tag)
    elif tag == "res":
        canvas.create_image(scrw*62/100, scrh*2/4, anchor=tk.N, image=black_box,  tag=tag)
        canvas.create_text(
            scrw*62/100,
            scrh*(2/4+3/100),
            text=text[:15] + "\n" + text[15:30] + "\n" + text[30:45] + "\n" + text[45:60],
            fill="white",
            font=("HG丸ｺﾞｼｯｸM-PRO", canvas.winfo_height()//40, "bold"),
            anchor="n",
            tag=tag)
    print(messageCount)
    canvas.update_idletasks()

In [8]:
#GUIセットアップ
from tkinter import *
from PIL import Image, ImageTk, ImageOps
import tkinter as tk
from tkinter.font import Font

# メインウィンドウ生成
root = tk.Tk()
#root.overrideredirect(True)  # ウィンドウバーを消す#windows用
root.iconphoto(False, tk.PhotoImage(file='icon.png'))#by 捨て犬A様 # ウィンドウアイコン設定
root.title("ナースロボ＿タイプＴ")
#root.attributes('-fullscreen', True)#フルスクリーン

root.config(bg="black") # 背景を黒にする#windows用
#root.attributes("-transparentcolor", "black") # 黒を透過する#windows用
#Linuxではウィンドウの

# ディスプレイサイズを取得してスクリーンのサイズを決める
scrh = int(root.winfo_screenheight()*2/3)
scrw = int(scrh*3/2)
root.geometry(f"{scrw}x{scrh}+{root.winfo_screenwidth()-scrw}+{root.winfo_screenheight()*95//100-scrh}")#windows用
#root.geometry(f"{scrw}x{scrh}+{root.winfo_screenwidth()-scrw}+{root.winfo_screenheight()-scrh}")#Linux用
#root.geometry(f"{scrw}x{scrh}+{root.winfo_screenwidth()-scrw}+{root.winfo_screenheight()//2-scrh}")#DualLinux用
#root.geometry(f"{scrw}x{scrh}+0+0")

# ウィンドウを左右に分ける
# frame_full = tk.Frame(root)
# frame_full.pack(fill=tk.BOTH, expand=True)
# frame_left = tk.Frame(root)
# frame_left.pack(side=tk.LEFT, expand=True)
# frame_right = tk.Frame(root)
# frame_right.pack(side=tk.RIGHT, expand=True)

# 画像を読み込む
nurse_o = Image.open("TT_Tachie_o.png")#by 捨て犬A様
nurse_n = Image.open("TT_Tachie_n.png")#by 捨て犬A様
alpha = Image.open("alpha.png")
back = Image.open("icon.png")#by 捨て犬A様
black = Image.open("black.png")
box = Image.open("massagebox.png")
black_box = Image.open("massagebox_black.png")
entryBox = Image.open("entryBox.png")
send = Image.open("send.png")
send = send.convert('RGBA')

# 画像のサイズを調整
nurse_height = int(scrh) # 画像の高さをディスプレイの1/3の大きさに指定
nurse_width = int(nurse_height*1500/1927) # 高さに横幅を合わせる
box_height = int(scrh/5)
box_width = int(box_height*1600/500)
entryBox_width = int(scrw*55/100)
entryBox_height = int(entryBox_width*400/2700*5/6)
send_height = int(entryBox_height*80/100)

# 画像をリサイズ
nurse_o = nurse_o.resize((nurse_width, nurse_height))
nurse_n = nurse_n.resize((nurse_width, nurse_height))
back = back.resize((scrw, scrh))
alpha = alpha.resize((scrw, scrh))
black = black.resize((scrw, scrh))
box = box.resize((box_width, box_height))
black_box = black_box.resize((box_width, box_height))
entryBox = entryBox.resize((entryBox_width, entryBox_height))
send = send.resize((send_height, send_height))

# ImageTkオブジェクトを作成
nurse_o = ImageTk.PhotoImage(nurse_o)
nurse_n = ImageTk.PhotoImage(nurse_n)
back = ImageTk.PhotoImage(back)
alpha = ImageTk.PhotoImage(alpha)
black = ImageTk.PhotoImage(black)
#box = ImageTk.PhotoImage(box)
black_box = ImageTk.PhotoImage(black_box)
entryBox = ImageTk.PhotoImage(entryBox)
send = ImageTk.PhotoImage(send)

# キャンバスを作成
#canvas = tk.Canvas(frame_full, width=scrw, height=scrh, bg='#101010', highlightthickness=0)
canvas = tk.Canvas(root, width=scrw, height=scrh, bg='black', highlightthickness=0)
canvas.pack()

# Canvasを作成して画像を表示
canvas.create_image(scrw/2, scrh/2, image=back, tag="back")
canvas.create_image(scrw/2, scrh/2, image=alpha, tag="alpha")
canvas.create_image(-scrw/20, 0, anchor=tk.NW, image=nurse_o)
nurse = canvas.create_image(-scrw/20, 0, anchor=tk.NW, image=nurse_n, tags="n_tag")
canvas.create_image(scrw*62/100, 0, anchor=tk.N, image=black_box, tag="res")
canvas.create_image(scrw*72/100, scrh*89/100, anchor=tk.N, image=entryBox)

text="あなたの言葉を読んだり聞いたりできます。\nあなたのAPIキーを教えてください。"
canvas.create_text(scrw*62/100, scrh*5/100,text=text[:15] + "\n" + text[15:30] + text[30:45] + "\n" + text[45:60], fill="white", font=("HG丸ｺﾞｼｯｸM-PRO", scrh//40, "bold") , anchor="n", tag = "res")#windows用
#text = canvas.create_text(scrw*62/100, scrh*5/100,text=text[:13] + "\n" + text[13:26] + "\n" + text[26:], fill="white", font=("HG丸ｺﾞｼｯｸM-PRO", scrh//40, "bold") , anchor="n", tag = "res")#Linux用

# font = ('HG丸ｺﾞｼｯｸM-PRO', 14)
# label = tk.Label(root, text='ボタンを押してください', font=font, bd=0, highlightthickness=0)
# label.configure(bg=root['bg'])
# label.place(x=scrw*62//100, y=0)

def key_handler(e):
    if e.keycode == 13 :#Enterキーでwindows用
    #if e.keycode == 36 :#EnterキーでLinux用
        clicked_button(canvas)#入力を完了する
    elif e.keycode == 27:#9Escキーでwindows用
    #elif e.keycode == 9:#9EscキーでLinux用
        root.destroy()#ウィンドウを終了する
# # Entryウィジェットをメインウィンドウに生成&配4
entry = Entry(root, fg="white", width=23, bg="#232323", bd=0, highlightthickness=0, font=("HG丸ｺﾞｼｯｸM-PRO", entryBox_height*3//10, "bold"))
entry.place(x=scrw*47/100, y=scrh*92/100)
entry.bind("<KeyPress>", key_handler)

'2592910511232key_handler'

In [9]:
import string
def is_half_width(text):#imputTextが半角か否かを判定する関数
    half_width_chars = string.ascii_letters + string.digits + string.punctuation
    for char in text:
        if char not in half_width_chars:
            return False
    return True

# ボタンの動作関数
def clicked_button(widget):
    global setting
    global role
    global log
    entryText=entry.get()# 入力ボックスの文字列を読み込み
    entry.delete(0, "end")# 入力ボックスの内容をクリア
    if entryText == "" and openai.api_key != "":#APIキーが入力されるまでは音声認識しない
        #音声認識部
        while True:
            inputText=record_audio()# 音声入力
            #print(inputText)
            #inputText = inputText.replace('　', '')  # 全角スペースを削除
            if inputText == "タイムアウトしました。":
                pass
            elif inputText == "音声を理解できませんでした。":
                break
            elif inputText == "音声認識サービスでエラーが発生しました。":
                break
            else:# 正しく音声認識できた時
                print("OK")
                break
            canvas.update_idletasks()
    else :#APIキーが空かテキスト入力された時
        inputText = entryText
    UpdateMessageLog(inputText, "input")#ユーザーの発言内容表示
    # 応答検討部
    if setting == True:#セッティングモードの時
        role = inputText
        setting = False
        res = "人格設定を変更しました。"
    else:# セッティングモードでないとき
        if inputText == "/setting":
    #         canvas = tk.Canvas(root, width=scrw, height=scrh, bg='black', highlightthickness=0)
    #         canvas.pack()
    #         canvas.itemconfigure("alpha", state="hidden")
    #         canvas.itemconfigure("back", state="hidden")
    #         root.overrideredirect(True)  # ウィンドウバーを消す#windows用
    #         root.attributes("-transparentcolor", "black") # 黒を透過する#windows用
    #         canvas.update_idletasks()
            res = "私の人格の設定文を入力してください。ただし、会話のたびに設定文の長さ分トークンを消費するのでご注意ください。"
            setting = True#セッティングモードを開始
        elif inputText == "":#inputTextが空でAPIキーも未入力の時
            res = "APIキーを入力してください"
        elif openai.api_key == "":# APIキーが未入力の時inutTextの中身はAPIキーの候補
            if is_half_width(inputText):
                openai.api_key = inputText# APIキーを読み込む
                res = ""# APIキー検証のためにクリア
                try:
                    res = openai.ChatCompletion.create(
                        model="gpt-3.5-turbo",
                        messages=[{"role": "user","content": inputText},],
                        max_tokens=1,
                    )
                    res = "APIキーを認識しました。"
                    print(res)
                except openai.error.AuthenticationError as error:
                    openai.api_key = ""# 有効ではないAPIキーをクリア
                    res = "そのAPIキーは有効ではありません。再度入力をお願いします。"
                    print(res)            
                print("半角文字です")
            else:
                print("半角文字ではありません")
                res = "APIキーは半角です。"
        elif inputText == "音声を理解できませんでした。":
            res = "ノイズが大きいか発音が不明瞭なため、何と言っているか聞き取れませんでした。"
        elif inputText == "音声認識サービスでエラーが発生しました。":
            res = "外部音声認識サービスへのアクセスに失敗しました。"
        else:#まったくエラーが無く、APIキーがあるとき
            #対話生成部
            res = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system","content": role},
                    #{"role": "assistant","content": log},#前回の対話を引き継ぎ
                    {"role": "user","content": inputText},
                ],
            )
            res = res["choices"][0]["message"]["content"]
            # ここまで--------------------------------
            print(res)
            log += inputText
            log += res
            #print(log)
    #         disp_text = res# 対話内容表示
    #         text = tk.Label(frame_right, text=disp_text, anchor=tk.NE, wraplength=scrw/4)
    #         text.pack()
            #time.sleep(1)
    #        res = inputText#ボイチェン（アニメーション動作確認用）    
    
    UpdateMessageLog(res, "res")#アシスタントの発言内容表示
    #音声合成部
    speaker = 47#14
    generate_wav(res,speaker)
    play_wav()
    print("会話終了")#標準出力もできる

In [10]:
role = "ご主人様を好きなメイドとして、ご主人様に対する返答をしてください。"
setting = False
log = ""
root.mainloop()

In [11]:
#!pyinstaller nurse-robo.py --onefile

#!pyinstaller nurse-robo.spec
